In [1]:
import urllib.request
import os
from tqdm import tqdm
from time import sleep
import math
import pdfplumber
from nltk.tokenize import word_tokenize
from datetime import datetime
from utils.gpl_tsdae import GplTsdae
import numpy as np

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList



2023-05-06 14:12:49.975134: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-06 14:12:50.585535: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-05-06 14:12:50.585584: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-05-06 14:12:50.585589: W tensorfl

In [2]:
gpl = GplTsdae()

[2023-05-06 14:12:51] INFO [sentence_transformers.SentenceTransformer.__init__:66] Load pretrained SentenceTransformer: ./models/gpl/TSDAE/500000


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[2023-05-06 14:12:53] INFO [models.gpl.gpl.toolkit.sbert.load_sbert:68] Set max_seq_length=350
[2023-05-06 14:12:53] INFO [beir.retrieval.search.dense.faiss_search._load:39] Loading Faiss ID-mappings from path: ./models/gpl/embedding/TSDAE/my-index.flat.tsv
[2023-05-06 14:12:53] INFO [beir.retrieval.search.dense.faiss_search._load:46] Loading Faiss Index from path: ./models/gpl/embedding/TSDAE/my-index.flat.faiss


In [3]:
tokenizer = AutoTokenizer.from_pretrained("StabilityAI/stablelm-tuned-alpha-3b")
model = AutoModelForCausalLM.from_pretrained("StabilityAI/stablelm-tuned-alpha-3b")
model.half().cuda()

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50688, 4096)
    (layers): ModuleList(
      (0): GPTNeoXLayer(
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attention): GPTNeoXAttention(
          (rotary_emb): RotaryEmbedding()
          (query_key_value): Linear(in_features=4096, out_features=12288, bias=True)
          (dense): Linear(in_features=4096, out_features=4096, bias=True)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=4096, out_features=16384, bias=True)
          (dense_4h_to_h): Linear(in_features=16384, out_features=4096, bias=True)
          (act): GELUActivation()
        )
      )
      (1): GPTNeoXLayer(
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=Tr

In [4]:
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        stop_ids = [50278, 50279, 50277, 1, 0]
        for stop_id in stop_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

In [5]:
def download_paper_and_save_as_txt(paper_id):
    if os.path.exists(f"papers_txt/{paper_id}.txt"):
        return
    for try_count in range(6):
        try:
            urllib.request.urlretrieve(f"https://export.arxiv.org/pdf/{paper_id}.pdf", f"./temp/{paper_id}.pdf")
            break
        except Exception as e:
            if try_count >= 5:
                raise e
            print(f"{try_count}-retry, get {paper_id} in 10 seconds.")
            sleep(10)
    pdfp = pdfplumber.open(f"./temp/{paper_id}.pdf")
    full_text = '\n'.join([page.extract_text() for page in pdfp.pages])
    tok = sent_tokenize(full_text)
    new_tok = []
    for s in tok:
        score = sum([ c not in "+-*/=^(){}[]0123456789!@ " and ord(c) < 128 for c in s ]) / (len(s))
        if score >= 0.8:
            new_tok.append(s)
    with open(f"papers_txt/{paper_id}.txt", "w") as f:
        f.write(' '.join(new_tok))
    os.remove(f"./temp/{paper_id}.pdf")

In [6]:
download_paper_and_save_as_txt('2010.15778')

In [7]:
!find papers_txt/13* | tail -10

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
papers_txt/1301.3834.txt
papers_txt/1301.7521.txt
papers_txt/1302.1727.txt
papers_txt/1302.3921.txt
papers_txt/1302.7145.txt
papers_txt/1303.2579.txt
papers_txt/1303.2580.txt
papers_txt/1303.5751.txt
papers_txt/1303.5768.txt
papers_txt/1304.1235.txt


In [8]:
query = "Is any part of p2p video streaming has same concept as bitcoin?"
paper_id = "2010.15778"
wc = 100

In [9]:
with open(f"papers_txt/{paper_id}.txt", "r") as f:
    paper_txt = f.read()

In [10]:
paper_ln = word_tokenize(paper_txt)
paper_ln = [' '.join(paper_ln[i:i+wc]) for i in range(0, len(paper_ln), wc)]
len(paper_ln)

18

In [11]:
paper_ln[0]

'Contextual BERT : Conditioning the Language Model Using a Global State TimoI.Denk AnaPeleteiroRamallo ZalandoSE ZalandoSE Berlin , Germany Berlin , Germany timo.denk @ zalando.de ana.peleteiro.ramallo @ zalando.de Abstract BERT is a popular language model whose main pre-training task is to ﬁll in the blank , i.e. , predicting a word that was masked out of a sentence , based on the remaining words . In some applications , however , havinganadditionalcontextcanhelpthemodelmaketherightprediction , e.g. , bytakingthedomainorthetimeofwritingintoaccount.ThismotivatesustoadvancetheBERTarchitecturebyaddingaglobalstateforconditioningonaﬁxed-sizedcontext.Wepresentourtwonovelapproachesandapplythemtoanindustryuse-case , wherewecompletefashionoutﬁtswithmissingarticles , conditionedonaspeciﬁccustomer.Anexperimentalcomparisontoothermethodsfromtheliteratureshowsthatourmethodsimprovepersonalizationsigniﬁcantly . 0202 tcO 92 1 Introduction ] LC.sc [ Sinceitspublication , theBERTmodelbyDevlinetal. ( 201

In [12]:
corpus = gpl.sbert.encode_corpus([ {'title': '', 'text': t} for t in paper_ln ])
corpus.shape

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

(18, 768)

In [13]:
%%time
scores = np.sum(np.repeat(gpl.sbert.encode_queries([query]), corpus.shape[0], axis=0) * corpus, axis=1)
scores

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 13.2 ms, sys: 526 µs, total: 13.8 ms
Wall time: 12.8 ms


array([10.306605, 14.606485, 14.237837, 14.721432, 13.958326, 11.095677,
       13.142616, 10.071073, 13.536208, 13.422296, 13.326892, 17.454824,
        9.566545, 15.631086,  9.853745,  8.704063, 10.138346, 11.51439 ],
      dtype=float32)

In [14]:
%%time
raw_input = sorted(sorted(list(zip(scores, enumerate(paper_ln))), key=lambda x: -x[0])[:4], key=lambda x: x[1][0])
raw_input

CPU times: user 28 µs, sys: 0 ns, total: 28 µs
Wall time: 32.4 µs


[(14.606485,
  (1,
   'itiscommonlypre-trainedonlargeamountsofunlabeleddata , andsubsequentlyﬁne-tunedonatargettask.Duringbothstages , themodel ’ sonlyinputisavariably-sizedsequenceofwords.Thereareuse-cases , however , wherehavinganadditionalcontextcanhelpthemodel.Consideraqueryintentclassiﬁerwhosesoleinputisauser ’ stextquery.Undertheassumptionthatusersfromdifferentagegroupsandprofessionsexpressthesameintentindifferentways , theclassiﬁerwouldbeneﬁtfromhavingaccesstothatusercontextinadditiontothequery.Alternatively , onemightconsidertrainingmultiplemodelsonseparate , agegroup-andprofession-speciﬁcsamples.However , thisapproachdoesnotscalewell , requiresmoretrainingdata , anddoesnotshareknowledgebetweenthemodels.Tothebestofourknowledge , thereisashortcomingineffectivemethodsforconditioningBERTonaﬁxed-sizedcontext.Motivatedbythis , andinspiredbythegraph-networksperspectiveonself-attentionmodels ( Battagliaetal.,2018 ) , weadvanceBERT ’ sarchitecturebyaddingaglobalstatethatenablesconditio

In [15]:
real_input = [ t[1][1] for t in raw_input]
real_input

['itiscommonlypre-trainedonlargeamountsofunlabeleddata , andsubsequentlyﬁne-tunedonatargettask.Duringbothstages , themodel ’ sonlyinputisavariably-sizedsequenceofwords.Thereareuse-cases , however , wherehavinganadditionalcontextcanhelpthemodel.Consideraqueryintentclassiﬁerwhosesoleinputisauser ’ stextquery.Undertheassumptionthatusersfromdifferentagegroupsandprofessionsexpressthesameintentindifferentways , theclassiﬁerwouldbeneﬁtfromhavingaccesstothatusercontextinadditiontothequery.Alternatively , onemightconsidertrainingmultiplemodelsonseparate , agegroup-andprofession-speciﬁcsamples.However , thisapproachdoesnotscalewell , requiresmoretrainingdata , anddoesnotshareknowledgebetweenthemodels.Tothebestofourknowledge , thereisashortcomingineffectivemethodsforconditioningBERTonaﬁxed-sizedcontext.Motivatedbythis , andinspiredbythegraph-networksperspectiveonself-attentionmodels ( Battagliaetal.,2018 ) , weadvanceBERT ’ sarchitecturebyaddingaglobalstatethatenablesconditioning.Withourproposedm

In [16]:

system_prompt = """<|SYSTEM|># StableLM Tuned (Alpha version)
- StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
- StableLM is excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
- StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and make jokes.
- StableLM will refuse to participate in anything that could harm a human.
"""

prompt = f"{system_prompt}<|USER|>{query}\n{' '.join([f'{i}) {t}' for i, t in enumerate(real_input)])} <ans> <|ASSISTANT|>"
# print(prompt)

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
tokens = model.generate(
  **inputs,
  max_new_tokens=256,
  temperature=0.7,
  do_sample=True,
  stopping_criteria=StoppingCriteriaList([StopOnTokens()]),
  top_p = 0.95, top_k = 50, early_stopping = False
)

print(tokenizer.decode(tokens[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


# StableLM Tuned (Alpha version)
- StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
- StableLM is excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
- StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and make jokes.
- StableLM will refuse to participate in anything that could harm a human.
Is any part of p2p video streaming has same concept as bitcoin?
0) itiscommonlypre-trainedonlargeamountsofunlabeleddata, andsubsequentlyﬁne-tunedonatargettask.Duringbothstages, themodel ’ sonlyinputisavariably-sizedsequenceofwords.Thereareuse-cases, however, wherehavinganadditionalcontextcanhelpthemodel.Consideraqueryintentclassiﬁerwhosesoleinputisauser ’ stextquery.Undertheassumptionthatusersfromdifferentagegroupsandprofessionsexpressthesameintentindifferentways, theclassiﬁerwouldbeneﬁtfromhavingaccesstothatusercontextinadditiontothequery.Alt

In [17]:
ans = tokenizer.decode(tokens[0], skip_special_tokens=True)

In [18]:
ans.split('<ans>')[-1]

' The paper discusses the concept of using a large amount of unlabeled data to train a machine learning model, such as a BERT model, which is used in various applications such as text classification, sentiment analysis, and image recognition. The authors propose using a large amount of unlabeled data to train a model, and then using this data to perform inference on the target model. The paper also proposes using a large amount of unlabeled data to train a model on a specific task or problem, which can lead to better performance. The paper also discusses the challenges of using unlabeled data to train a model, such as the lack of sufficient data and the need for a large amount of computation. The paper proposes using a large amount of unlabeled data to train a model on a specific task or problem to improve performance. The paper also discusses the use of unlabeled data in natural language processing, such as text classification, sentiment analysis, and image recognition. Overall, the p

In [19]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Sat May  6 14:13:31 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.86.01    Driver Version: 515.86.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:01:00.0 Off |                  Off |
| 32

In [20]:

system_prompt = """<|SYSTEM|># StableLM Tuned (Alpha version)
- StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
- StableLM is excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
- StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and make jokes.
- StableLM will refuse to participate in anything that could harm a human.
"""

prompt = f"{system_prompt}<|USER|>Give 3 short search quries that You think I would like to search next from this context.\n BERT is a popular language model whose main pre-training task is to fill in the blank, i.e., predicting a word that was masked out of a sentence, based on the remaining words. In some applications, however, having an additional context can help the model make the right prediction, e.g., by taking the domain or the time of writing into account. This motivates us to advance the BERT architecture by adding a global state for conditioning on a fixed-sized context. We present our two novel approaches and apply them to an industry use-case, where we complete fashion outfits with missing articles, conditioned on a specific customer. An experimental comparison to other methods from the literature shows that our methods improve personalization significantly. BERT is a cutting-edge language representation model pre-trained by a large corpus, which achieves superior performances on various natural language understanding tasks. However, a major blocking issue of applying BERT to online services is that it is memory-intensive and leads to unsatisfactory latency of user requests, raising the necessity of model compression. Existing solutions leverage the knowledge distillation framework to learn a smaller model that imitates the behaviors of BERT. However, the training procedure of knowledge distillation is expensive itself as it requires sufficient training data to imitate the teacher model. In this paper, we address this issue by proposing a hybrid solution named LadaBERT (Lightweight adaptation of BERT through hybrid model compression), which combines the advantages of different model compression methods, including weight pruning, matrix factorization and knowledge distillation. LadaBERT achieves state-of-the-art accuracy on various public datasets while the training overheads can be reduced by an order of magnitude. <ans> <|ASSISTANT|>"
# print(prompt)

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
tokens = model.generate(
  **inputs,
  max_new_tokens=256,
  temperature=0.7,
  do_sample=True,
  stopping_criteria=StoppingCriteriaList([StopOnTokens()]),
  top_p = 0.95, top_k = 50, early_stopping = False
)

print(tokenizer.decode(tokens[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


# StableLM Tuned (Alpha version)
- StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
- StableLM is excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
- StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and make jokes.
- StableLM will refuse to participate in anything that could harm a human.
Give 3 short search quries that You think I would like to search next from this context.
 BERT is a popular language model whose main pre-training task is to fill in the blank, i.e., predicting a word that was masked out of a sentence, based on the remaining words. In some applications, however, having an additional context can help the model make the right prediction, e.g., by taking the domain or the time of writing into account. This motivates us to advance the BERT architecture by adding a global state for conditioning on a fixed-sized con

In [21]:

system_prompt = """<|SYSTEM|># StableLM Tuned (Alpha version)
- StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
- StableLM is excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
- StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and make jokes.
- StableLM will refuse to participate in anything that could harm a human.
"""

# prompt = f"{system_prompt}<|USER|>Gives 5 most important fact from context that relate to this query in form of number 1 to 5 list \"What is bert\"\n BERT is a popular language model whose main pre-training task is to fill in the blank, i.e., predicting a word that was masked out of a sentence, based on the remaining words. In some applications, however, having an additional context can help the model make the right prediction, e.g., by taking the domain or the time of writing into account. This motivates us to advance the BERT architecture by adding a global state for conditioning on a fixed-sized context. We present our two novel approaches and apply them to an industry use-case, where we complete fashion outfits with missing articles, conditioned on a specific customer. An experimental comparison to other methods from the literature shows that our methods improve personalization significantly. BERT is a cutting-edge language representation model pre-trained by a large corpus, which achieves superior performances on various natural language understanding tasks. However, a major blocking issue of applying BERT to online services is that it is memory-intensive and leads to unsatisfactory latency of user requests, raising the necessity of model compression. Existing solutions leverage the knowledge distillation framework to learn a smaller model that imitates the behaviors of BERT. However, the training procedure of knowledge distillation is expensive itself as it requires sufficient training data to imitate the teacher model. In this paper, we address this issue by proposing a hybrid solution named LadaBERT (Lightweight adaptation of BERT through hybrid model compression), which combines the advantages of different model compression methods, including weight pruning, matrix factorization and knowledge distillation. LadaBERT achieves state-of-the-art accuracy on various public datasets while the training overheads can be reduced by an order of magnitude. \n <ans> <|ASSISTANT|>"

prompt = f"{system_prompt}<|USER|>Gives 5 most important fact from context that relate to this query in form of number 1 to 5 list \"What is bert\"\n BERT is a popular language model whose main pre-training task is to fill in the blank, i.e., predicting a word that was masked out of a sentence, based on the remaining words. In some applications, however, having an additional context can help the model make the right prediction, e.g., by taking the domain or the time of writing into account. This motivates us to advance the BERT architecture by adding a global state for conditioning on a fixed-sized context. We present our two novel approaches and apply them to an industry use-case, where we complete fashion outfits with missing articles, conditioned on a specific customer. An experimental comparison to other methods from the literature shows that our methods improve personalization significantly. BERT is a cutting-edge language representation model pre-trained by a large corpus, which achieves superior performances on various natural language understanding tasks. However, a major blocking issue of applying BERT to online services is that it is memory-intensive and leads to unsatisfactory latency of user requests, raising the necessity of model compression. Existing solutions leverage the knowledge distillation framework to learn a smaller model that imitates the behaviors of BERT. However, the training procedure of knowledge distillation is expensive itself as it requires sufficient training data to imitate the teacher model. In this paper, we address this issue by proposing a hybrid solution named LadaBERT (Lightweight adaptation of BERT through hybrid model compression), which combines the advantages of different model compression methods, including weight pruning, matrix factorization and knowledge distillation. LadaBERT achieves state-of-the-art accuracy on various public datasets while the training overheads can be reduced by an order of magnitude. \n <ans> <|ASSISTANT|>"

# print(prompt)

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
tokens = model.generate(
  **inputs,
  max_new_tokens=256,
  temperature=0.7,
  do_sample=True,
  stopping_criteria=StoppingCriteriaList([StopOnTokens()]),
  top_p = 0.95, top_k = 50, early_stopping = False
)

print(tokenizer.decode(tokens[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


# StableLM Tuned (Alpha version)
- StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
- StableLM is excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
- StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and make jokes.
- StableLM will refuse to participate in anything that could harm a human.
Gives 5 most important fact from context that relate to this query in form of number 1 to 5 list "What is bert"
 BERT is a popular language model whose main pre-training task is to fill in the blank, i.e., predicting a word that was masked out of a sentence, based on the remaining words. In some applications, however, having an additional context can help the model make the right prediction, e.g., by taking the domain or the time of writing into account. This motivates us to advance the BERT architecture by adding a global state for conditioni

In [22]:

system_prompt = """<|SYSTEM|># StableLM Tuned (Alpha version)
- StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
- StableLM is excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
- StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and make jokes.
- StableLM will refuse to participate in anything that could harm a human.
"""

# prompt = f"{system_prompt}<|USER|>Gives 5 most important fact from context that relate to this query in form of number 1 to 5 list \"What is bert\"\n BERT is a popular language model whose main pre-training task is to fill in the blank, i.e., predicting a word that was masked out of a sentence, based on the remaining words. In some applications, however, having an additional context can help the model make the right prediction, e.g., by taking the domain or the time of writing into account. This motivates us to advance the BERT architecture by adding a global state for conditioning on a fixed-sized context. We present our two novel approaches and apply them to an industry use-case, where we complete fashion outfits with missing articles, conditioned on a specific customer. An experimental comparison to other methods from the literature shows that our methods improve personalization significantly. BERT is a cutting-edge language representation model pre-trained by a large corpus, which achieves superior performances on various natural language understanding tasks. However, a major blocking issue of applying BERT to online services is that it is memory-intensive and leads to unsatisfactory latency of user requests, raising the necessity of model compression. Existing solutions leverage the knowledge distillation framework to learn a smaller model that imitates the behaviors of BERT. However, the training procedure of knowledge distillation is expensive itself as it requires sufficient training data to imitate the teacher model. In this paper, we address this issue by proposing a hybrid solution named LadaBERT (Lightweight adaptation of BERT through hybrid model compression), which combines the advantages of different model compression methods, including weight pruning, matrix factorization and knowledge distillation. LadaBERT achieves state-of-the-art accuracy on various public datasets while the training overheads can be reduced by an order of magnitude. \n <ans> <|ASSISTANT|>"

prompt = f"{system_prompt}<|USER|>Give me the overview of “What is GAN” from given context in one paragraph. \n Generative Adversarial Network (GAN) is a well known computationally complex algorithm requiring signficiant computational resources in software implementations including large amount of data to be trained. This makes its implementation in edge devices with conventional microprocessor hardware a slow and difficult task. In this paper, we propose to accelerate the computationally intensive GAN using memristive neural networks in analog domain. We present a fully analog hardware design of Deep Convolutional GAN (DCGAN) based on CMOS-memristive convolutional and deconvolutional networks simulated using 180nm CMOS technology. The Generative Adversarial Network (GAN) is a powerful and flexible tool that can generate high-fidelity synthesized data by learning. It has seen many applications in simulating events in High Energy Physics (HEP), including simulating detector responses and physics events. However, training GANs is notoriously hard and optimizing their hyperparameters even more so. It normally requires many trial-and-error training attempts to force a stable training and reach a reasonable fidelity. Significant tuning work has to be done to achieve the accuracy required by physics analyses. This work uses the physics-agnostic and high-performance-computer-friendly hyperparameter optimization tool HYPPO to optimize and examine the sensitivities of the hyperparameters of a GAN for two independent HEP datasets. This work provides the first insights into efficiently tuning GANs for Large Hadron Collider data. We show that given proper hyperparameter tuning, we can find GANs that provide high-quality approximations of the desired quantities. We also provide guidelines for how to go about GAN architecture tuning using the analysis tools in HYPPO. Generative adversarial networks (GAN) are a class of powerful machine learning techniques, where both a generative and discriminative model are trained simultaneously. GANs have been used, for example, to successfully generate “deep fake” images. A recent trend in malware research consists of treating executables as images and employing image-based analysis techniques. In this research, we generate fake malware images using auxiliary classifier GANs (AC-GAN), and we consider the effectiveness of various techniques for classifying the resulting images. Our results indicate that the resulting multiclass classification problem is challenging, yet we can obtain strong results when restricting the problem to distinguishing between real and fake samples. While the AC-GAN generated images often appear to be very similar to real malware images, we conclude that from a deep learning perspective, the AC-GAN generated samples do not rise to the level of deep fake malware images. \n <ans> <|ASSISTANT|>"

# print(prompt)

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
tokens = model.generate(
  **inputs,
  max_new_tokens=256,
  temperature=0.7,
  do_sample=True,
  stopping_criteria=StoppingCriteriaList([StopOnTokens()]),
  top_p = 0.95, top_k = 50, early_stopping = False
)

print(tokenizer.decode(tokens[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


# StableLM Tuned (Alpha version)
- StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
- StableLM is excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
- StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and make jokes.
- StableLM will refuse to participate in anything that could harm a human.
Give me the overview of “What is GAN” from given context in one paragraph. 
 Generative Adversarial Network (GAN) is a well known computationally complex algorithm requiring signficiant computational resources in software implementations including large amount of data to be trained. This makes its implementation in edge devices with conventional microprocessor hardware a slow and difficult task. In this paper, we propose to accelerate the computationally intensive GAN using memristive neural networks in analog domain. We present a fully analog